In [56]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


Словарь направлений

In [3]:
direction_names = {
    2: "Информатика и вычислительная техника",
    3: "Программная инженерия",
    4: "Информационная безопасность",
    5: "Биология",
    6: "Математика и математическое моделирование"
}

Очистка датасета

In [49]:
# Загрузка данных из CSV файла
data = pd.read_csv('education_directions.csv')

# Удаление дубликатов
data.drop_duplicates(inplace=True)

# Добавление новых признаков средних значений
data['mean'] = data.mean(axis=1)

# Вывод первых нескольких строк для проверки
print(data.head())

# Сохранение измененного датасета в новый CSV файл
data.to_csv('education_directions.csv', index=False)

   direction  math  informatics   geometry   algebra   physics  chemistry  \
0          2     1            1          1         0         0          0   
1          2     1            1          0         0         0          0   
2          2     0            1          1         0         0          0   
3          2     1            0          1         0         0          0   
4          2     1            1          0         0         1          0   

   Unnamed: 7  biology   geography      mean  
0           0        0         NaN  0.555556  
1           0        0         NaN  0.444444  
2           0        0         NaN  0.444444  
3           0        0         NaN  0.444444  
4           1        0         NaN  0.666667  


In [50]:
# Загрузка данных из CSV файла
data = pd.read_csv('education_directions.csv')

# Проверка распределения данных
print(data['direction'].value_counts())

# Подготовка данных
X = data.drop('direction', axis=1).apply(lambda row: ' '.join(row.astype(str)), axis=1)
y = data['direction']

# Кодирование направлений в числовые метки
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.6, random_state=42)

direction
2    21
4    18
6    18
3    17
5    10
Name: count, dtype: int64


Создание и обучение модели

In [51]:
def train_model(X_train, y_train):
    model = make_pipeline(CountVectorizer(token_pattern=r'(?u)\b\w+\b'), RandomForestClassifier(random_state=42))
    model.fit(X_train, y_train)
    return model

In [52]:
model = train_model(X_train, y_train)

# Оценка модели
accuracy = model.score(X_test, y_test)
print("Точность модели на тестовой выборке:", accuracy)

Точность модели на тестовой выборке: 0.803921568627451


In [62]:
def predict_direction(model, label_encoder, subjects, direction_names):
    text_representation = ' '.join(map(str, subjects))
    prediction_encoded = model.predict([text_representation])
    prediction_decoded = label_encoder.inverse_transform(prediction_encoded)
    predicted_direction = direction_names[prediction_decoded[0]]
    return predicted_direction

# Пример ввода любимых предметов
favorite_subjects = [1, 1, 0, 0, 0, 0, 1, 0]  # Математика, информатика, физика

# Получение предсказания
predicted_direction = predict_direction(model, label_encoder, favorite_subjects, direction_names)
print("Рекомендованное направление:", predicted_direction)

Рекомендованное направление: Информационная безопасность


In [57]:
import joblib

In [58]:
joblib.dump(model, 'model.joblib')

['model.joblib']